## Enabling GoogleAuth for Fast API

## Integrating FastAPI with Google Authentication

Integrating FastAPI with Google Authentication involves using Google’s OAuth 2.0 protocol to allow users to log in to your FastAPI application using their Google credentials. Here are the general steps to achieve this integration:

### 1. Create a Project in Google Cloud Console
1. Go to the [Google Cloud Console](https://console.cloud.google.com/) and create a new project if you haven’t already.
2. Enable the “Google+ API” in the **APIs & Services > Dashboard** section.

### 2. Create OAuth 2.0 Credentials
1. In the Google Cloud Console, go to **APIs & Services > Credentials**.
2. Click on **Create Credentials** and select **OAuth client ID**.
3. Choose **Web application** as the application type.
4. Add your FastAPI application’s URL (e.g., `http://localhost:8000`) to the **Authorized redirect URIs**.

### 3. Install Required Libraries
Install the necessary Python libraries using pip:


In [ ]:
%pip install fastapi
%pip install uvicorn
%pip install python-jose[cryptography]
%pip install python-multipart
%pip install requests

## Implementing Google Authentication in FastAPI

Below is a simple example of how you can integrate Google Authentication with FastAPI:

In [ ]:
from fastapi import FastAPI, Depends
from fastapi.security import OAuth2PasswordBearer
import requests
from jose import jwt

app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Replace these with your own values from the Google Developer Console
GOOGLE_CLIENT_ID = "your-google-client-id"
GOOGLE_CLIENT_SECRET = "your-google-client-secret"
GOOGLE_REDIRECT_URI = "your-google-redirect-uri"

@app.get("/login/google")
async def login_google():
    return {
        "url": f"https://accounts.google.com/o/oauth2/auth?response_type=code&client_id={GOOGLE_CLIENT_ID}&redirect_uri={GOOGLE_REDIRECT_URI}&scope=openid%20profile%20email&access_type=offline"
    }

@app.get("/auth/google")
async def auth_google(code: str):
    token_url = "https://accounts.google.com/o/oauth2/token"
    data = {
        "code": code,
        "client_id": GOOGLE_CLIENT_ID,
        "client_secret": GOOGLE_CLIENT_SECRET,
        "redirect_uri": GOOGLE_REDIRECT_URI,
        "grant_type": "authorization_code",
    }
    response = requests.post(token_url, data=data)
    access_token = response.json().get("access_token")
    user_info = requests.get("https://www.googleapis.com/oauth2/v1/userinfo", headers={"Authorization": f"Bearer {access_token}"})
    return user_info.json()

@app.get("/token")
async def get_token(token: str = Depends(oauth2_scheme)):
    return jwt.decode(token, GOOGLE_CLIENT_SECRET, algorithms=["HS256"])

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

This example provides two endpoints: `/login/google` for initiating the Google login flow and `/auth/google` for exchanging the authorization code for an access token. The user's information can be retrieved from Google's userinfo endpoint. Additionally, it includes an endpoint `/token` to decode and verify the JWT token obtained after successful authentication.

Remember to replace `your-google-client-id`, `your-google-client-secret`, and `your-google-redirect-uri` with the appropriate values from your Google Cloud Console project.

### Run the FastAPI Application
Save the code in a file (e.g., `main.py`) and run the FastAPI application using Uvicorn:

```bash
uvicorn main:app --reload


With these steps, your FastAPI application should be integrated with Google Authentication, allowing users to log in using their Google accounts. Please note that this is a simple example, and in a real-world scenario, you would need to handle error cases, store user information securely, and possibly add database integration for user management.